In [ ]:
import random
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from matplotlib.colors import LinearSegmentedColormap
import os


def initial_spins(L):
    return np.random.choice([1, -1], size=(L,L))

def calculate_energy(spins, J, h, L):
    energy = 0
    for i in range(L):
        for j in range(L):
            energy -= h*spins[i, j]
            energy -= J*(spins[(i+1)%L, j] + spins[i, (j+1)%L])*spins[i,j]
    return energy

def magnetization(spins, L):
    net_spin = 0
    for i in range(L):
        for j in range(L):
            net_spin += spins[i, j]
    return net_spin/(L**2)

def metropolis_algorithim(spins, beta, J, h, L):
    #making a trial flip
    i, j = np.random.randint(0, L, size=2)
    site = spins[i, j]
    trial_flip = site * -1
    trial_flip_spins = spins.copy()
    trial_flip_spins[i, j] = trial_flip

    #calculating old energy
    old_energy = -h*spins[i, j] - J*(spins[(i+1)%L, j] + spins[i, (j+1)%L]+
                                     spins[(i - 1) % L, j] +  spins[i, (j-1)%L])*spins[i, j]
    #calculating new energy
    new_energy = -h*trial_flip_spins[i, j] - J*(trial_flip_spins[(i+1)%L, j] + trial_flip_spins[i, (j+1)%L]+
                                     trial_flip_spins[(i - 1) % L, j] +  trial_flip_spins[i, (j-1)%L])*trial_flip_spins[i, j]
    #computing delta E and deciding if we should accept the change
    delta_energy = new_energy - old_energy

    if delta_energy <= 0:
        spins[i, j] = trial_flip
    else:
        w = np.exp(-beta * delta_energy)
        r = random.random()
        if r <= w:
            spins[i,j] = trial_flip

    return(spins)

def monte_carlo(beta, J, h, L, steps, spins):
    energies = [calculate_energy(spins, J, h, L)]
    mag_values = [magnetization(spins, L)]
    for step in tqdm(range(steps)):
        spins = metropolis_algorithim(spins, beta, J, h, L)
        if step % 100 == 0 or step==steps-1:
            energy = calculate_energy(spins, J, h, L)
            energies.append(energy)

            mag_value = magnetization(spins, L)
            mag_values.append(mag_value)

        if step % 100 == 0 or step==steps-1:
            # Visualization
            fig = plt.figure(figsize=(12, 12))
            
            ax0 = fig.add_subplot(3, 2, 1)
            scatter_2D(copy_of_spins_initial, ax0, "Initial Lattice Configuration")
            
            ax1 = plt.subplot(3, 2, 2)
            scatter_2D(spins, ax1, "Final Lattice Configuration")
            
            ax2 = plt.subplot(3, 2, 3)
            scatter_2D(spins - copy_of_spins_initial, ax2, "Residual Configuration")
            
            plt.subplot(3, 2, 4)
            plt.title("Energy vs. Monte Carlo Steps")
            plt.plot(range(0, len(energies) * 100, 100), energies, label="Energy", color='darkorange')
            plt.xlabel("Monte Carlo Step")
            plt.ylabel("Energy")
            plt.legend()
            
            plt.tight_layout()
            plt.savefig(str(step).zfill(5)+'.png', format='png')

    return spins, energies, mag_values

In [ ]:
def normalize(array):
    normalized_array = array.copy()
    max = np.max(array)
    min = np.min(array)
    for i in range(len(array)):
        normalized_array[i] = (array[i] - min)/(max-min)
    return normalized_array

def difference(array):
    differences = array.copy()[:-1]
    for i in range(len(array)-1):
        differences[i] = array[i+1] - array[i]
    return differences


def scatter_2D(data, ax, title):
    x = np.indices(data.shape)[0]
    y = np.indices(data.shape)[1]
    col = data.flatten()

    p2d = ax.imshow(data, cmap='PiYG', vmin=-1, vmax=1)
    plt.colorbar(p2d, label="Spin")
    ax.set_title(title)
    return

In [ ]:
J = 1
h = -1
L = 25
T = 10**(-5)
steps = 10000
beta = 1/T

spins_initial = initial_spins(L)
copy_of_spins_initial = spins_initial.copy()
final_spins, energies, mag_values = monte_carlo(beta, J, h, L, steps, spins_initial)

# Visualization
fig = plt.figure(figsize=(12, 12))

ax0 = fig.add_subplot(3, 2, 1)
scatter_2D(copy_of_spins_initial, ax0, "Initial Lattice Configuration")

ax1 = plt.subplot(3, 2, 2)
scatter_2D(final_spins, ax1, "Final Lattice Configuration")

ax2 = plt.subplot(3, 2, 3)
scatter_2D(final_spins - copy_of_spins_initial, ax2, "Residual Configuration")

plt.subplot(3, 2, 4)
plt.title("Energy vs. Monte Carlo Steps")
plt.plot(range(0, len(energies) * 100, 100), energies, label="Energy", color='mediumvioletred')
plt.xlabel("Monte Carlo Step")
plt.ylabel("Energy")
plt.legend()

plt.tight_layout()
plt.show()
plt.close()

In [ ]:
# make GIF
make_gif = 'magick -delay 20 -loop 0 *.png test.gif'
os.system(make_gif)

# delete images when finished
delete_images = 'rm *.png'
os.system(delete_images)
plt.close('all')